In [2]:
import pandas as pd

In [3]:
# load csv file
file_path = './raw-results/testdata.csv'
df = pd.read_csv(file_path)

# remove headers
df_dropped = df.drop(index=[0, 1, 2])

# separate question blocks
TLX = df_dropped.iloc[:, 0:20]
WUS = df_dropped.iloc[:, 20:30]
recommend = df_dropped.iloc[:, 30:31]
free_text_q1 = df_dropped.iloc[:, 31:32]
free_text_q2 = df_dropped.iloc[:, 32:33]

In [4]:
TLX

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Task 1 TLX_1,Task 1 TLX_2,Task 1 TLX_3
3,2024-08-01 21:06:41,2024-08-01 21:06:41,Survey Test,NaN,100,0,True,2024-08-01 21:06:41,R_bHNBywsJ9M5wtVk,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Neither agree nor disagree,Strongly disagree,Disagree
4,2024-08-01 21:06:41,2024-08-01 21:06:41,Survey Test,NaN,100,0,True,2024-08-01 21:06:41,R_blsKsSMoxPx4nlA,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Agree,Somewhat agree,Strongly agree
5,2024-08-01 21:06:41,2024-08-01 21:06:41,Survey Test,NaN,100,0,True,2024-08-01 21:06:41,R_9BUbdaOAui027pc,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Disagree,Neither agree nor disagree,Somewhat agree
6,2024-08-01 21:06:41,2024-08-01 21:06:41,Survey Test,NaN,100,0,True,2024-08-01 21:06:41,R_9BLsEgBciPqKRKu,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Strongly agree,Neither agree nor disagree,Somewhat disagree
7,2024-08-01 21:06:41,2024-08-01 21:06:41,Survey Test,NaN,100,0,True,2024-08-01 21:06:42,R_7a2wedzCfin3uWa,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Disagree,Somewhat agree,Somewhat agree
8,2024-08-01 21:06:42,2024-08-01 21:06:42,Survey Test,NaN,100,0,True,2024-08-01 21:06:42,R_byMUNhI22HorK7Q,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Strongly agree,Disagree,Disagree
9,2024-08-01 21:06:42,2024-08-01 21:06:42,Survey Test,NaN,100,0,True,2024-08-01 21:06:42,R_4Ix1WwDQ8RpQE62,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Somewhat disagree,Strongly disagree,Somewhat agree
10,2024-08-01 21:06:42,2024-08-01 21:06:42,Survey Test,NaN,100,0,True,2024-08-01 21:06:42,R_9YzUKNxtqxtUD7E,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Neither agree nor disagree,Disagree,Somewhat agree
11,2024-08-01 21:06:42,2024-08-01 21:06:42,Survey Test,NaN,100,0,True,2024-08-01 21:06:43,R_6D7OnHVj4nIj4eG,NaN,NaN,NaN,NaN,NaN,NaN,test,NaN,Somewhat agree,Somewhat disagree,Strongly agree


In [159]:
#-------------------------------------------
# Analysis for TLX
#-------------------------------------------

# function to map 7-point likert scale to 0-100
def map_to_0_100(likert_point):
    return ((likert_point - 1) / 6) * 100

# hardcoding since this is quicker
indices_to_exclude = [1, 5, 9, 13, 17]  # for question 2: "I was successful in accomplishing what I wanted to do"
reverse_code_columns = [i for i in range(TLX.shape[1]) if i not in indices_to_exclude]

# reverse code columns and scake to 0-100
TLX_int = TLX.astype(int)
TLX_int.iloc[:, reverse_code_columns] = TLX_int.iloc[:, reverse_code_columns].apply(lambda x: 8-x)
TLX_scaled=TLX_int.iloc[:, :].map(map_to_0_100)

# calculate mean for each task
# note: for each group, we are taking the mean of the mean of each column
# This gives the correct mean since the number of elements of all groups is the same
# https://math.stackexchange.com/questions/95909/why-is-an-average-of-an-average-usually-incorrect
num_groups = len(TLX_scaled.columns) // 4
for i in range(num_groups):
    group = TLX_scaled.iloc[:, i*4:(i+1)*4]
    print("Mean TLX Score for Task {task_num}:".format(task_num=i+1), 
          group.mean().mean())

# calculate mean for all tasks
print("Mean TLX Score for All Tasks:", TLX_scaled.mean().mean())

Mean TLX Score for Task 1: 69.6969696969697
Mean TLX Score for Task 2: 71.21212121212122
Mean TLX Score for Task 3: 70.45454545454545
Mean TLX Score for Task 4: 68.18181818181819
Mean TLX Score for Task 5: 73.4848484848485
Mean TLX Score for All Tasks: 70.60606060606062


In [122]:
#-------------------------------------------
# Analysis for WUS
# Guidelines: https://measuringu.com/sus/
#-------------------------------------------

WUS_int = WUS.astype(int)
odd_items = [i for i in range(WUS.shape[1]) if i % 2 == 0]      # index starts at 0
even_items = [i for i in range(WUS.shape[1]) if i % 2 != 0]

# for odd items, subtract one from the user response
WUS_int.iloc[:, odd_items] = WUS_int.iloc[:, odd_items].apply(lambda x: x-1)

# for even-numbered items, subtract the user responses from 5
WUS_int.iloc[:, even_items] = WUS_int.iloc[:, even_items].apply(lambda x: 5-x)

# ensure that all values are between 0 and 4 after scaling
assert ((WUS_int >= 0) & (WUS_int <= 4)).all().all()

# add up the converted responses for each user and multiply that total by 2.5
row_sums = WUS_int.sum(axis=1) * 2.5

# ensure that all values are between 0 and 100 after scaling
assert ((row_sums >= 0) & (row_sums <= 100)).all()

# find the average WUS score
print("Mean WUS Score:", row_sums.mean())

Mean WUS Score: 64.77272727272727


In [133]:
#-------------------------------------------
# How likely are you to recommend this website to others?
# Scale: 0 to 10
#-------------------------------------------

recommend_int = recommend.astype(int)
print("Mean Recommendation Score (all participants):", recommend_int.mean().mean())

# if we remove the two non-responses (0)
drop_nan = recommend_int[recommend_int != 0]
print("Mean Recommendation Score (drop NaN):", drop_nan.mean().mean())

Mean Recommendation Score (all participants): 5.818181818181818
Mean Recommendation Score (drop NaN): 7.111111111111111


In [134]:
free_text_q1

,FreeTextQ1
3,NaN
4,Efficient
5,The system's pretty intuitive and I was able t...
6,Simplicity
7,NaN
8,NaN
9,NaN
10,I really liked the UI
11,The UI was very aesthetically pleasing and smo...
12,Consistent design


In [135]:
free_text_q2

,FreeTextQ2
3,NaN
4,Interface can be much simpler
5,I'm not sure. Some usability features could be...
6,Clearer differentiation between accessing and ...
7,functions is clicking everywhere on the web pa...
8,NaN
9,NaN
10,I would say that creating and grading exams co...
11,"When going on it for the first time, it was a ..."
12,Add alerts
